## How to use Data Lab Services From Web Apps

*Revised:  May 17, 2017*

This notebook documents how to call the various Data Lab services (e.g. AuthManager, QueryManager, etc) from Web applications by directly accessing the service URL.  It will explain which arguments are required, when an authorization token is required, the return value of the service (and how it might change depending on the arguments or auth token) and how errors may be trapped.  Unless otherwise noted, all specified arguments are required for each service.

All Data Lab services are implemented as a RESTful web service and may be accessed using HTTP GET or POST protocols.  The service endpoints may be accessed from a variety of languages and tools.  The intent of this notebook is to describe the endpoint URL and behavior of the service more than how to access that service from any particular framework.


In [221]:
# Standard notebook imports
from __future__ import print_function
from urllib2 import Request, urlopen, URLError, HTTPError
import urllib
import os, sys

# Define the base service URLs
AUTH    = "http://dldev.datalab.noao.edu/auth"           # Auth Manager
QUERY   = "http://dldev.datalab.noao.edu/query"         # Query Manager
STORAGE = "http://dldev.datalab.noao.edu/storage"       # Storage Manager
RES     = "http://dldev.datalab.noao.edu/res"           # Resource Manager

def HTTP_CALL (mode, url, args, headers=None):
    """  Demo code to call an return a service URL.
    """
    data = (urllib.urlencode(args) if args is not None else "")
    try:
        if mode == "GET":
            svc_url = ((url + data) if args is not None else url)
            req = urllib2.Request(svc_url, headers=headers)
        else:
            req = urllib2.Request(url, data, headers=headers)
        response = urllib2.urlopen(req)
    except HTTPError as e:
        print ('Error code: %d: %s' % (e.code, e.reason))
    except URLError as e:
        print ('We failed to reach a server.\nReason: %s' % e.reason)
    else:
        if mode == 'GET':
            print ('GET:  %s\n\nRESPONSE:\n%s\n%s' % (response.geturl(), response.info(), response.read()))
        else:
            print ('POST:  %s\n\t%s\n\nRESPONSE\n%s\n%s' % (response.geturl(), req.get_data(), response.info(), response.read()))

---

## <u>Auth Manager Services</u>

### authClient.login (user, password=None, debug=False, verbose=False)

<b>Description:</b>  The *login()* method is used to obtain an authorization token from the Data Lab to access a specific user's resources (virtual storage, compute processing, etc).  That token must be passed to other methods as needed.

<b>Returns:</b>
    - On success, a valid user authorization token
    - On error, the error message explaining the failure
    - A HTML header element in the response to set a cookie containing the auth token

In [ ]:
# Example 0:  A valid user login
args = []
args.append( {"username" : "demo00",      # account name     (required)
              "password" : "balatad",     # account password (required)
              "profile" : "default",      # service profile  (optional)
              "verbose" : False}          # verbose output   (optional, not used)
           )

# Example 1: Anonymous user login
args.append ( {"username" : "anonymous"} )

# Example 2: Invalid username error
args.append ( {"username" : "Barney",
               "password" : "Rubble"}
            )

# Example 3: Invalid password error
args.append ( {"username" : "demo00",
               "password" : "xyzzy"}
            )

headers = {}                        # No token needed

example = 0                         # Example number to run
HTTP_CALL ("GET", AUTH + "/login?", args[example], headers=headers)

### authClient.isAlive (svc_url=DEF_SERVICE_URL)

<b>Description:</b>  The *isAlive()* method is used to determine whether the AuthManager service is online and responding.  This is essentially a call to the base Auth Manager service URL.

<b>Returns:</b>
    - On success, a 'Hello World' message from the Auth Manager
    - On error, the HTTP error code

In [ ]:
# Example 0:  A valid service, no arguments are required
args = []
args.append("")
           
# Example 1: An invalid URL
args.append ( {"svc_url" : "http://foo.far.com/auth"} )

headers = {}                        # No token needed

example = 0                         # Example number to run
HTTP_CALL ("GET", AUTH, args[example], headers=headers)

### authClient.isValidToken (token)

<b>Description:</b>  The isValidToken() method is used to determine whether the given token represents a valid user currently logged into the Data Lab system.

<b>Returns:</b>
    - a 'True' or 'False' string depending on token validity

In [ ]:
# Example 0: A valid anonymous user token
args = []
args.append ( {"token" : "anonymous.0.0.anon_access"} )

# Example 1:  An invalid user token
args.append ( {"token" : "foo.0.0.bar"} )

headers = {}                        # No token needed

example = 0
HTTP_CALL ("GET", AUTH + "/isValidToken?", args[example], headers=headers)

### authClient.isValidUser (user)

<b>Description:</b>  The *isValidUser()* method is used to determine whether the specified username is valid in the system.

<b>Returns:</b>
    - a 'True' or 'False' string depending on username validity

In [ ]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"user" : "anonymous"} )

# Example 1:  An invalid user token
args.append ( {"user" : "foobar"} )

headers = {}                        # No token needed

example = 0
HTTP_CALL ("GET", AUTH + "/isValidUser?", args[example], headers=headers)

### authClient.isValidPassword (user, password)

<b>Description:</b>  The *isValidPassword()* method is used to determine whether the given password is valid for the named user.

<b>Returns:</b>
    - a 'True' or 'False' string depending on password validity

In [ ]:
# Example 0: A valid anonymous user
args = []
args.append ( {"user" : "anonymous",
               "password" : "anonymous"}
            )

# Example 1:  An invalid user password
args.append ( {"user" : "anonymous",
               "password" : "xyzzy"}
            )

headers = {}                        # No token needed

example = 0
HTTP_CALL ("GET", AUTH + "/isValidPassword?", args[example], headers=headers)

### authClient.isUserLoggedIn (user)

<b>Description:</b>  The *isUserLoggedIn()* method is used to determine whether the named user is logged-in to the Data Lab system.

<b>Returns:</b>
    - a 'True' or 'False' string depending on login status

In [192]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"user" : "anonymous"} )

headers = {}                      # No token needed

example = 0
HTTP_CALL ("GET", AUTH + "/isUserLoggedIn?", args[example], headers=headers)

http://dldev.datalab.noao.edu/auth/isUserLoggedIn?user=anonymous

Server: nginx/1.10.2
Date: Thu, 18 May 2017 05:16:38 GMT
Content-Type: text/html; charset=utf-8
Content-Length: 5
Connection: close
Access-Control-Allow-Origin: *

False


### authClient.isTokenLoggedIn (token)

<b>Description:</b>  The *isTokenLoggedIn()* method is used to determine whether the given token is logged-in to the Data Lab system.

<b>Returns:</b>
    - a 'True' or 'False' string depending on login status

In [ ]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"token" : "anonymous.0.0.anon_access"} )

headers = {}                        # No token needed

example = 0
HTTP_CALL ("GET", AUTH + "/isTokenLoggedIn?", args[example], headers=headers)

### authClient.logout (token)

<b>Description:</b>  The *logout()* method is used to logout of the Data Lab.

<b>Returns:</b>
    - On success, a "OK" response string
    
<b>NOTE:</b>  An 'INTERNAL SERVER ERROR' is expected for the anonymous token in the example, use a real token for testing.

In [ ]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"token" : "anonymous.0.0.anon_access"} )

headers = {}                        # No token needed

example = 0
HTTP_CALL ("GET", AUTH + "/logout?", args[example], headers=headers)

### authClient.hasAccess (user, resource)

<b>Description:</b>  The *hasAccess()* method is used to determine whether the named user has access to the specified resource.

<b>Returns:</b>
    - On success, a "OK" response string

In [ ]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"user" : "anonymous",
               "resource" : "vos://"}
            )

# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'anonymous.0.0.anon_access'}

example = 0
HTTP_CALL ("GET", AUTH + "/hasAccess?", args[example], headers=headers)

### authClient.passwordReset (token, username, password)

<b>Description:</b>  The *passwordReset()* method is used to change a user's password in the system.  For this to work, the user must be logged in an present a valid token for the current session as well as their username and the new password string.

<b>Returns:</b>
    - On success, a valid token for the new password
    - On failure, the error message
    
<b>NOTE:</b>  An 'UNAUTHORIZED' is expected for the anonymous token in the example, use a real token for testing.

In [ ]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"username" : "anonymous",
               "password" : "newpass"} 
            )

# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'anonymous.0.0.anon_access'}

example = 0
HTTP_CALL ("GET", AUTH + "/passwordReset?", args[example], headers=headers)

***
## <u>Query Manager Services</u>


### queryClient.query (token, adql=None, sql=None, fmt='csv', out=None, async=False)

<b>Description:</b>  The *query()* method is used to submit a data query to the Data Lab.  Anonymous queries may be submitted provided the output is not saved to VOSpace or MyDB.

<b>Arguments:</b>
    - token   A valid user token (or NULL)
    - adql    An ADQL query string (submits to TAP service) (optional)
    - sql     An SQL query string (submits directly to the database) (optional)
    - ofmt     Output format (csv, ascii, votable, or fits) (optional)
    - out     Output destination
    - async   ASynchronous query if set
    
Only one of 'adql' or 'sql' may be specified.  If the 'async' flag is set, results will need to be retrieved by a separate call to the *results()* service once the job is complete.

<b>Returns:</b>
    - An "OK" string on success if the output is saved elsewhere
    - The results of the query if 'out' is null and 'async' is False
    - a jobID string if the 'async' flag is set

In [223]:
# Example 0: An ADQL query that returns results directly.
# 'out' can be specified as "vos://<filename>" to save the
# results to VOSpace.  Likewise, a "mydb://<tablename>"
# value will save to a table in MyDB.
args = []
args.append ( {"adql" : "select top 10 * from usno.a2",
               "ofmt" : "csv", 
               "out" : None,
               "async" : False  
              }
            )

# Example 1: An SQL query that returns results directly.
args.append ( {"sql" : "select * from usno.a2 limit 10",
               "ofmt" : "csv",
               "out" : None,
               "async" : False
              }
            )

# Example 2: An ASync ADQL query, this returns a jobID you can pass
# to the /status and /results service.
args.append ( {"adql" : "select top 5 * from usno.a2",
               "ofmt" : "csv",
               "out" : None,
               "async" : True
              }
            )

# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'anonymous.0.0.anon_access'}

example = 2
method = "POST"                      # Both GET and POST are accepted
HTTP_CALL (method, QUERY + "/query?", args[example], headers=headers)

POST:  http://dldev.datalab.noao.edu/query/query?
	ofmt=csv&async=True&adql=select+top+5+%2A+from+usno.a2&out=None

RESPONSE
Server: nginx/1.10.2
Date: Thu, 18 May 2017 17:08:22 GMT
Content-Type: text/html; charset=utf-8
Content-Length: 16
Connection: close
Access-Control-Allow-Origin: *

nj78tk2uzjvvwi9h


### queryClient.status (token, jobId)

<b>Description:</b>  The *status()* method is used to determine the status of the specified asynchronous job.

<b>Returns:</b>
    - The job status of the specified 'jobId'.  Results may be retrieved once the status is 'COMPLETED'
    
<b>NOTE:</b>  A valid async jobId is required, this example won't actually run

In [233]:
# Example 0: A valid jobID
args = []
args.append ( {"jobid" : "nj78tk2uzjvvwi9h"} )
 
# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'anonymous.0.0.anon_access'}

example = 0
HTTP_CALL ("GET", QUERY + "/status?", args[example], headers=headers)

Error code: 400: BAD REQUEST


### queryClient.results (token, jobId)

<b>Description:</b>  The *results()* method is used to retrieve the results of a completed async job.

<b>Returns:</b>
    - results of the async job

In [234]:
# Example 0: A valid jobID
args = []
args.append ( {"jobid" : "nj78tk2uzjvvwi9h"} )
 
# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'anonymous.0.0.anon_access'}

example = 0
HTTP_CALL ("GET", QUERY + "/results?", args[example], headers=headers)

Error code: 400: BAD REQUEST


### queryClient.list (token, table="")

<b>Description:</b>  The *isTokenLoggedIn()* method is used to list a user's MyDB tables.

<b>Returns:</b>
    - a list of table names if no table name is specified
    - a list of table columns if a specific table is provided

In [235]:
# Example 0: A listing of all tables
args = []
args.append ( {"table" : ""} )

# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'dldemo.9999.9999.demo_access'}

example = 0
HTTP_CALL ("GET", AUTH + "/list?", args[example], headers=headers)

Error code: 404: NOT FOUND


### queryClient.drop (token, table, profile="")

<b>Description:</b>  The *drop()* method is used to drop a table from the user's MyDB.

<b>Returns:</b>
    - Status 204 if the table is deleted
    - An exception of error

In [ ]:
# Example 0: A valid anonymous user 
args = []
args.append ( {"table" : "test",
               "profile" : "default"} 
            )

# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'dldemo.9999.9999.demo_access'}

example = 0
HTTP_CALL ("GET", AUTH + "/delete?", args[example], headers=headers)

## <u>Storage Manager Services</u>

`
def get(token, fr, to, verbose=True):
def put(token, fr, to, verbose=True):
def cp(token, fr, to, verbose=False):
def ln(token, fr, target):
def ls(token, name, format='csv'):
def mkdir(token, name):
def mv(token, fr, to, verbose=False):
def rm(token, name, verbose=False):
def rmdir(token, name):
def saveAs(token, data, name):
def tag(token, name, tag):
`

## <u>Resource Manager Services</u>

`
def createUser (username, password, email, name):
def passwordReset (password):
def readUser (keyword):
def updateUser (keyword, value):
def deleteUser (username, password):
def createGroup (groupName):
def readGroup (keyword):
def updateGroup (keyword, value):
def deleteGroup (groupName):
def createResource (resource):
def readResource (keyword):
def updateResource (keyword, value):
def deleteResource (resource):
`

### resClient.createUser (username, password, email, name)

<b>Description:</b>  The *createUser()* method is used to create a User in the Data Lab, however it does not activate the user account until a moderator has approved the account.

<b>Returns:</b>
    - On success, an "OK" string
    - On failure, an error message
    
<b>NOTE:</b>  The service URL in this case contains a hard-coded argument.

In [238]:
# Example 0: A test user account
args = []
args.append ( {"username" : "test00",
               "password" : "testpw",
               "email" : "test@test.com",
               "name" : "Test UserName"
              }
            )

# Valid auth token required in header
headers = {}

example = 0
HTTP_CALL ("GET", RES + "/create?what=user&", args[example], headers=headers)

Error code: 400: BAD REQUEST


### resClient.passwordReset (token, username, password)

<b>Description:</b>  The *passwordReset()* method is used to reset a user's account password.

<b>Returns:</b>
    - On success, an "OK" string
    - On failure, an error message
    
<b>NOTE:</b>  Not fully implemented.

In [238]:
# Example 0: A test user account
args = []
args.append ( {"username" : "test00",
               "password" : "testpw",
               "email" : "test@test.com",
               "name" : "Test UserName"
              }
            )

# Valid auth token required in header
headers = { 'X-DL-AuthToken' : 'dldemo.9999.9999.demo_access'}

example = 0
HTTP_CALL ("GET", RES + "/passwordReset", args[example], headers=headers)

Error code: 400: BAD REQUEST
